In [1]:
filenames = [
    'O2 Grouper Pipeline - Task List.xlsx',
    'P2 v3 Authorization_DS manager - list of tasks.xlsx',
    'P2 v3 Replatforming Tasks List.xlsx',
    # 'P2 testing tasks.xlsx',
]


In [2]:
import tools.dev_o2_excel_ingestor as o2

for filename in filenames:
    print("Ingesting: ", filename)
    rdf = o2.generate_rdf_from_xlsx('./tools/', filename)
    o2.send_rdf_to_dgraph('localhost:9080', rdf)
    print("Finished ingestion: ", filename)



Ingesting:  O2 Grouper Pipeline - Task List.xlsx
!!! FINISHED !!!
Finished ingestion:  O2 Grouper Pipeline - Task List.xlsx
Ingesting:  P2 v3 Authorization_DS manager - list of tasks.xlsx
!!! FINISHED !!!
Finished ingestion:  P2 v3 Authorization_DS manager - list of tasks.xlsx
Ingesting:  P2 v3 Replatforming Tasks List.xlsx
!!! FINISHED !!!
Finished ingestion:  P2 v3 Replatforming Tasks List.xlsx


In [1]:
import tools.lib.pmtx_client.query_baselines as rb
import requests
import json
import pandas as pd


url = 'http://localhost:8080/graphql'


query_project_filter = {
    "or": [
        {"name": {"allofterms": "P2 v3 Replatforming Tasks List.xlsx"}},
        # {"name": {"allofterms": "O2 Grouper Pipeline - Task List.xlsx"}},
        # {"name": {"allofterms": "P2 v3 Authorization_DS manager - list of tasks.xlsx"}}
    ]
}


normalized_baseline = rb.request_and_normalize_baselines_from_pmtx(url, query_project_filter)
dfp, dfd = rb.baseline_to_pandas_df(normalized_baseline)



query = '''
    query ($projects_ids:[ID!]) {
        queryProject (filter: {and: [{id: $projects_ids} ]}) {
            id
            customFields
        }
    }
'''

r = requests.post(url=url, json={"query": query, "variables": {"projects_ids": dfp.project_id.to_list()}})

project_details = r.json()['data']['queryProject']

for project in project_details:
    if project['customFields'] is not None:
        details = json.loads(project['customFields'])
        project['sprint'] = (int(details['sprint'].split(' ')[1]) if details['sprint'].split(' ')[1].isdigit() else None) if 'sprint' in details else None
    else: 
        project['sprint'] = None
    del project['customFields']

project_details = pd.DataFrame(project_details)
dfp = dfp.merge(project_details, how='left', left_on='project_id', right_on='id')
dfp.drop(columns=['id'], inplace=True)


print("Finished.")


Finished.


In [2]:
import tools.lib.task_assignee_estimators.solver_base as base

est = base.SolverBase(dfp,dfd)

est.config()
est.initialize()

est.find_incorrect_dependencies_FS()

# est.lp

# dfp[dfp.start.notnull() & dfp.finish.notnull()]

,finish_x,project_id,worktime,start,finish_y


In [3]:
from time import time
import pandas as pd

import tools.lib.task_assignee_estimators.primitive_estimation as estimator




solver = estimator.ProposeAssigment()

start_date = dfp.start.min()


projects = dfp.copy(deep=True)
dependencies = dfd.copy(deep=True)

solver.av = estimator.create_availability_calendar(number_of_users=9)
solver.av['project_id'] = None

algo_time_start = time()

for sprint in [6,7,8,9,10]:
    solver.projects = projects[(projects.sprint < sprint)]
    # print(solver.projects[solver.projects.sprint == 6].shape[0])
    solver.dependencies = dependencies
    solver.av[solver.av.project_id.notnull()]

    # # Calculation part
    solver.initialize(start_date)
    solver.av[solver.av.project_id.notnull()]
    # x1 = solver.av.copy(deep=True)

    # finish_date = solver.assign_projects_from_last_task_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)
    finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

    solver.av[solver.av.project_id.notnull()]
    # x2 = solver.av.copy(deep=True)

    solver.update_projects()
    solver.av[solver.av.project_id.notnull()]


    for index, row in solver.projects.iterrows(): # TODO: find better way to update
        projects.loc[projects.project_id == row['project_id'], 'start'] = solver.projects[solver.projects.project_id == row['project_id']].start.iloc[0]
        projects.loc[projects.project_id == row['project_id'], 'finish'] = solver.projects[solver.projects.project_id == row['project_id']].finish.iloc[0]


    break

# ## resolve rest (NULLs)
# solver.projects = projects
# # print(solver.projects[solver.projects.sprint == 6].shape[0])
# solver.dependencies = dependencies

# # # Calculation part
# solver.initialize(start_date)

# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

# solver.update_projects()

# # projects[(projects.sprint < sprint), 'start'] = solver.projects
# for index, row in solver.projects.iterrows(): # TODO: find better way to update
#     projects.loc[projects.project_id == row['project_id'], 'start'] = solver.projects[solver.projects.project_id == row['project_id']].start.iloc[0]
#     projects.loc[projects.project_id == row['project_id'], 'finish'] = solver.projects[solver.projects.project_id == row['project_id']].finish.iloc[0]


algo_time_finish = time()

print('Project finish timestamp: ' + str(finish_date))
print('Calculation time [s]: ' + str(algo_time_finish - algo_time_start))
print('Unassigned workers time during project: ' + str((solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].finish - solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].start).sum()))

print("Finished.")




d:\pmt_x\tools\lib\task_assignee_estimators\primitive_estimation.py:323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.lp.start[self.lp.start.isnull()] = partial_update_from
d:\pmt_x\tools\lib\task_assignee_estimators\primitive_estimation.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['project_id'] = None
d:\pmt_x\tools\lib\task_assignee_estimators\primitive_estimation.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [14]:
sprt = 6
print("min: ", solver.projects[solver.projects.sprint == sprt].start.min())
print("min: ", solver.projects[solver.projects.sprint == sprt].start.max())

min:  2021-04-20 11:30:00.317826+00:00
min:  2021-05-12 15:30:00.317826+00:00


In [5]:
import tools.lib.pmtx_client.mutate_baselines as mb

solver.projects.drop(columns=['sprint'], inplace=True)

root_id = "0x7bd"
baseline_name = "infinite_resources_10"
# baseline_name = "limited_8_sprint_5_only"


baseline_id = mb.add_baseline_and_return_id(url, baseline_name, root_id)
mb.add_project_baseline(url, solver.projects, baseline_id, root_id)
mb.modify_project_baseline_predecessors(url, solver.dependencies, baseline_id)
# mb.add_resource_baseline(url, solver.av, baseline_id)

print("Finished.")

d:\pmt_x\tools\lib\pmtx_client\mutate_baselines.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  children['parent_id'] = root_id
Finished.


In [9]:
solver.dependencies

,project_id,predecessor_id,type
0,0x69,0x9d,FS
1,0x137,0x12f,FS
2,0x137,0x9d,FS
3,0x141,0x137,FS
4,0x89,0x83,FS
...,...,...,...
395,0x4ed,0x632,FS
396,0x4ed,0x6ad,FS
397,0x2ce,0x6ce,FS
398,0x355,0x2ce,FS


In [2]:
from time import time
import pandas as pd

import tools.lib.task_assignee_estimators.primitive_estimation as estimator






solver = estimator.ProposeAssigment()

start_date = dfp.start.min()

solver.projects = dfp.copy(deep=True)
solver.dependencies = dfd.copy(deep=True)
# solver.av = pd.DataFrame([], columns=['user_id', 'start', 'finish'])
solver.av = estimator.create_availability_calendar(number_of_users=9)
# solver.av = expander.create_availability_frame()
solver.av['project_id'] = None
print(solver.av.shape[0])

# Calculation part

solver.initialize(start_date)


algo_time_start = time()


finish_date = solver.assign_projects_infinite_resources()
# finish_date = solver.assign_projects_to_resources_first_free(one_worker_per_project=True)
# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(one_worker_per_project=True)
# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

print(solver.av.shape[0])
solver.update_projects()

algo_time_finish = time()

print('Project finish timestamp: ' + str(finish_date))
print('Calculation time [s]: ' + str(algo_time_finish - algo_time_start))
# print('Unassigned workers time during project: ' + str((solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].finish - solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].start).sum()))

print("Finished.")

1926
1926
Project finish timestamp: 2021-03-04 05:13:00+00:00
Calculation time [s]: 1.0838160514831543
Finished.


In [38]:
solver.av[solver.av.project_id.notnull()]

,start,finish,user_id,project_id
0,2021-04-19 08:00:00.319256+00:00,2021-04-19 15:00:00.319256+00:00,0,0x551
1,2021-04-20 08:00:00.319256+00:00,2021-04-20 15:00:00.319256+00:00,0,0x12e
2,2021-04-21 08:00:00.319256+00:00,2021-04-21 16:00:00.319256+00:00,0,0x6bf
3,2021-04-22 08:00:00.319256+00:00,2021-04-22 16:00:00.319256+00:00,0,0x6bf
4,2021-04-23 08:00:00.319256+00:00,2021-04-23 09:00:00.319256+00:00,0,0x6bf
...,...,...,...,...
2148,2021-05-21 14:00:00.319256+00:00,2021-05-21 16:00:00.319256+00:00,0,0xb
2149,2021-05-24 10:00:00.319256+00:00,2021-05-24 14:00:00.319256+00:00,0,0xb1
2150,2021-05-25 11:30:00.319256+00:00,2021-05-25 12:00:00.319256+00:00,0,0xc9
2151,2021-05-24 14:00:00.319256+00:00,2021-05-24 16:00:00.319256+00:00,7,0x111
